### HTML 웹 크롤링 실습
- 제조업 PMI 데이터 수집
    - tradingeconomics 웹 서비스에서 아시아의 제조업 PMI 데이터를 수집하여 PMI 지표가 상승한 국가의 데이터를 출력
    - `https://ko.tradingeconomics.com/`
    - 지표 -> 사업 -> 제조업 PMI -> 아시아

In [1]:
import requests
from bs4 import BeautifulSoup

1. 아시아 국가별 PMI 지수 데이터 수집

In [3]:
url = 'https://ko.tradingeconomics.com/country-list/manufacturing-pmi?continent=asia'
response = requests.get(url)
response

<Response [200]>

In [70]:
dom = BeautifulSoup(response.content, 'html.parser')
elements = dom.select(".table-responsive > table > tr")
#ctl00_ContentPlaceHolder1_ctl01_UpdatePanel1 > div > div.table-responsive > table > tbody > tr:nth-child(1)
len(elements)

20

In [71]:
elements[0]


<tr class="datatable-row">
<td style="max-width: 120px; overflow: hidden; padding-left: 10px; text-align: left; font-weight: 600;">
<a href="/china/manufacturing-pmi">
                                        중국
                                    </a></td>
<td data-value="53.60">
                                    53.6
                                </td>
<td>53</td>
<td class="hidden-xs"><span>2020-10</span></td>
<td class="hidden-sm hidden-xs">
                                    포인트
                                </td>
</tr>

In [84]:
datas_ls = []
for element in elements:
    datas = {}
    datas["country"] = element.select('td')[0].text.strip()
    datas["last"] = element.select('td')[1].text.strip()
    datas["before"] = element.select('td')[2].text.strip()
    datas["date"] = element.select('td')[3].text.strip()
    datas_ls.append(datas)
    
datas_ls

[{'country': '중국', 'last': '53.6', 'before': '53', 'date': '2020-10'},
 {'country': '홍콩', 'last': '49.8', 'before': '47.7', 'date': '2020-10'},
 {'country': '인도', 'last': '58.9', 'before': '56.8', 'date': '2020-10'},
 {'country': '인도네시아', 'last': '47.8', 'before': '47.2', 'date': '2020-10'},
 {'country': '이스라엘', 'last': '53.5', 'before': '53.1', 'date': '2020-09'},
 {'country': '일본', 'last': '48.7', 'before': '47.7', 'date': '2020-10'},
 {'country': '카자흐스탄', 'last': '50.8', 'before': '52.6', 'date': '2020-10'},
 {'country': '레바논', 'last': '43.3', 'before': '42.1', 'date': '2020-10'},
 {'country': '말레이시아', 'last': '48.5', 'before': '49', 'date': '2020-10'},
 {'country': '미얀마', 'last': '30.6', 'before': '35.9', 'date': '2020-10'},
 {'country': '필리핀', 'last': '48.5', 'before': '50.1', 'date': '2020-10'},
 {'country': '카타르', 'last': '51.5', 'before': '51.4', 'date': '2020-10'},
 {'country': '사우디 아라비아', 'last': '51', 'before': '50.7', 'date': '2020-10'},
 {'country': '싱가포르', 'last': '50.5',

In [85]:
asia_pmi_df = pd.DataFrame(datas_ls)
asia_pmi_df.tail(5)

,country,last,before,date
15,스리랑카,59.8,57.9,2020-09
16,대만,55.1,55.2,2020-10
17,태국,50.8,49.9,2020-10
18,아랍 에미리트,49.5,51,2020-10
19,베트남,51.8,52.2,2020-10


,country,last,before,date
15,스리랑카,64.6,67.3,2020-07
16,대만,50.6,46.2,2020-07
17,태국,45.9,43.5,2020-07
18,아랍 에미리트,50.8,50.4,2020-07
19,베트남,47.6,51.1,2020-07


2. PMI 지수 변동률 컬럼 추가

In [86]:
asia_pmi_df[["last", "before"]] = asia_pmi_df[["last", "before"]].astype("float")
asia_pmi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  20 non-null     object 
 1   last     20 non-null     float64
 2   before   20 non-null     float64
 3   date     20 non-null     object 
dtypes: float64(2), object(2)
memory usage: 768.0+ bytes


In [91]:
asia_pmi_df["change"] = asia_pmi_df["last"] - asia_pmi_df["before"]
asia_pmi_df.tail(5)

,country,last,before,date,change
15,스리랑카,59.8,57.9,2020-09,1.9
16,대만,55.1,55.2,2020-10,-0.1
17,태국,50.8,49.9,2020-10,0.9
18,아랍 에미리트,49.5,51.0,2020-10,-1.5
19,베트남,51.8,52.2,2020-10,-0.4


,country,last,before,date,change
15,스리랑카,64.6,67.3,2020-07,-2.7
16,대만,50.6,46.2,2020-07,4.4
17,태국,45.9,43.5,2020-07,2.4
18,아랍 에미리트,50.8,50.4,2020-07,0.4
19,베트남,47.6,51.1,2020-07,-3.5


3. 제조업 PMI가 이전 달보다 안 좋아진 국가리스트 출력

In [89]:
asia_pmi_df[asia_pmi_df["change"]<0]

,country,last,before,date,change
6,카자흐스탄,50.8,52.6,2020-10,-1.8
8,말레이시아,48.5,49.0,2020-10,-0.5
9,미얀마,30.6,35.9,2020-10,-5.3
10,필리핀,48.5,50.1,2020-10,-1.6
16,대만,55.1,55.2,2020-10,-0.1
18,아랍 에미리트,49.5,51.0,2020-10,-1.5
19,베트남,51.8,52.2,2020-10,-0.4


,country,last,before,date,change
1,홍콩,44.5,49.6,2020-07,-5.1
2,인도,46,47.2,2020-07,-1.2
4,이스라엘,38.5,39.3,2020-05,-0.8
6,카자흐스탄,48,52.9,2020-07,-4.9
8,말레이시아,50,51,2020-07,-1.0
10,필리핀,48.4,49.7,2020-07,-1.3
15,스리랑카,64.6,67.3,2020-07,-2.7
19,베트남,47.6,51.1,2020-07,-3.5
